In [1]:
from gnomepy.backtest.backtest import *
from gnomepy.backtest.strategy import *
from gnomepy.backtest.signal import *

In [2]:
client = MarketDataClient(bucket="market-data-raw-dev", aws_profile_name="AWSAdministratorAccess-443370708724")
client_data_params = {
    "exchange_id": 1,
    "security_id": 3,
    "start_datetime": datetime.datetime(2025, 4, 7),
    "end_datetime": datetime.datetime(2025, 4, 28),
    "schema_type": SchemaType.MBP_10,
}

signal_names = {'rolling_mean_30': ['bidCount9'], 
                'rolling_median_40': ['bidCount9']}

signals = [{'signal': global_signals[signal_name], 'columns': signal_names[signal_name]} for signal_name in signal_names.keys()]

strat = Strategy(name="Simple Strategy", signals=signals, action=Action(lambda x: x))

backtest = Backtest(
    client=client,
    strategy=strat,
    exchange_id=client_data_params["exchange_id"],
    security_id=client_data_params["security_id"],
    start_datetime=client_data_params["start_datetime"],
    end_datetime=client_data_params["end_datetime"],
    schema_type=client_data_params["schema_type"]
)

Found 38 matching keys


In [3]:
output_df = backtest.run(data_type='pandas')

In [5]:
output_df

,exchangeId,securityId,timestampEvent,timestampSent,timestampRecv,price,size,action,side,flags,...,bidCount8,askCount8,bidPrice9,askPrice9,bidSize9,askSize9,bidCount9,askCount9,bidCount9_rolling_mean_30,bidCount9_rolling_median_40
0,1,3,2025-04-06 23:59:59.881000+00:00,NaT,2025-04-07 00:00:00.157522365+00:00,1579.3,22.1146,b'M',b'N',16,...,11,8,1578.4,1580.3,497.0772,92.7955,28,11,NaN,NaN
1,1,3,2025-04-07 00:00:00.007000+00:00,NaT,2025-04-07 00:00:03.280162207+00:00,1579.4,0.1372,b'T',b'B',16,...,11,8,1578.4,1580.3,497.0772,92.7955,28,11,NaN,NaN
2,1,3,2025-04-07 00:00:00.007000+00:00,NaT,2025-04-07 00:00:03.280162207+00:00,1579.5,1.4413,b'T',b'B',16,...,11,8,1578.4,1580.3,497.0772,92.7955,28,11,NaN,NaN
3,1,3,2025-04-07 00:00:00.141000+00:00,NaT,2025-04-07 00:00:03.280179440+00:00,1579.5,1.4413,b'M',b'N',16,...,21,13,1578.5,1580.4,228.0260,434.9203,12,22,NaN,NaN
4,1,3,2025-04-07 00:00:01.091000+00:00,NaT,2025-04-07 00:00:03.760934806+00:00,1579.4,0.0276,b'T',b'A',16,...,21,13,1578.5,1580.4,228.0260,434.9203,12,22,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886519,1,3,2025-04-07 18:59:57.155000+00:00,NaT,2025-04-07 18:59:57.346145741+00:00,1548.5,0.3059,b'M',b'N',16,...,16,4,1547.2,1549.1,187.4835,669.1080,24,15,18.700000,17.5
886520,1,3,2025-04-07 18:59:57.665000+00:00,NaT,2025-04-07 18:59:57.857009056+00:00,1548.5,0.3059,b'M',b'N',16,...,15,4,1547.2,1549.1,223.3975,302.8857,25,14,19.166667,18.0
886521,1,3,2025-04-07 18:59:58.211000+00:00,NaT,2025-04-07 18:59:58.465691805+00:00,1548.5,0.3059,b'M',b'N',16,...,15,6,1547.2,1549.1,223.3975,248.3787,25,13,19.600000,18.0
886522,1,3,2025-04-07 18:59:58.724000+00:00,NaT,2025-04-07 18:59:58.950249904+00:00,1548.5,0.3059,b'M',b'N',16,...,15,7,1547.2,1549.1,265.0140,298.7675,25,15,20.066667,18.5
